In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import SGD
import cv2
import skimage
import os
from imgaug.imgaug import augmenters as iaa
from densenet121 import DenseNet
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

from sklearn.cluster import MiniBatchKMeans

Using TensorFlow backend.


In [2]:
model = DenseNet(reduction=0.5, classes=1000, weights_path='./densenet121_weights_tf.h5')
print(model.layers[-1].output_shape)
model.layers.pop()
model.layers.pop()
output = model.layers[-1].output
new_model = Model(model.input, output)

densenet121.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:164: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
/home/wanlu/4032/venv/local/lib/pyt

densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv3_12_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:135: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="conv3_blk")`
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4

densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_21_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_22_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_22_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_23_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_23_x2

(None, 1000)


In [3]:
new_model.layers[-1].output_shape

(None, 1024)

In [4]:
train_features = np.load('../data/features_234000.npy')
train_labels = np.load('../data/features_labels_234000.npy')

In [5]:
# train_features.shape

In [6]:
def get_data(train_features, train_labels):
#     train_features, train_labels = shuffle(train_features, train_labels, random_state=7)
    
    label_to_features = {}
    label_to_indexes = {}
    for index, label in enumerate(train_labels):
        label = tuple(label)
        if label not in label_to_features:
            label_to_features[label] = [train_features[index]]
            label_to_indexes[label] = [index]
        else:
            label_to_features[label].append(train_features[index])
            label_to_indexes[label].append(index)
    labels = []
    img_features = []
    indexes = []
    for l in label_to_features.keys():
        if len(label_to_features[l]) % 100 == 0:
            split = len(label_to_features[l])/100
        else:
            split = len(label_to_features[l])/100 + 1
        for i in range(split):
            labels.append(l)
            # if it's the last part
            if i == split-1:
                img_features.append(np.asarray(label_to_features[l][i*100:]))
                indexes.append(np.asarray(label_to_indexes[l][i*100:]))
            # split into parts, each with 100 img featrues
            else:
                img_features.append(np.asarray(label_to_features[l][i*100:(i+1)*100]))
                indexes.append(np.asarray(label_to_indexes[l][i*100:(i+1)*100]))

    labels = np.asarray(labels)
    img_features = np.asarray(img_features)
    indexs = np.asarray(indexes)
#     biz_features = np.asarray([np.mean(f, axis=0) for f in img_features])
    
#     biz_features, img_features, labels, indexes = shuffle(biz_features, img_features, labels, indexes, random_state=7)
    
    return img_features, labels, indexes

In [7]:
def train(iteration, train_X, train_y):
    print "training"
    # classifier = OneVsRestClassifier(LinearSVC(random_state=0))
    lst_clfs = [LinearSVC() for i in range(9)]
    for i in range(iteration):
        print "Iteration: " + str(i)
        for k in range(9):
            clf = lst_clfs[k]
            part = train_X.shape[0]/5
            scores = []
            for fold in range(5):
                val_X = train_X[fold*part:(fold+1)*part]
                val_y = train_y[fold*part:(fold+1)*part]
                tr_X = np.vstack((train_X[0:fold*part], train_X[(fold+1)*part:]))
                tr_y = np.vstack((train_y[0:fold*part], train_y[(fold+1)*part:]))
                clf.fit(tr_X, tr_y[:, k])
                scores.append(clf.score(val_X, val_y[:, k]))
            print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(scores), np.std(scores) * 2))

    return lst_clfs

In [8]:
def test(lst_clfs, test_X, test_y):
    for k in range(9):
        clf = lst_clfs[k]
        s = clf.score(test_X, test_y[:,k])
        print str(k) + "'s classifier, test acc = " + str(round(s, 4))
    

In [9]:
def get_biz_features(img_features, labels, indexes, num_cluster, kmn):
    biz_features = np.zeros([len(labels),(1024*num_cluster)])

    for idx, features in enumerate(img_features):
        feature_index = indexes[idx]
        cluster_lable = np.array(kmn[feature_index])
        
        # for each biz group, mean feature vectore for those in the same cluster
        for kn in range(num_cluster):
            x = features[cluster_lable==kn]
            # if feature doesn't belong to any cluster,
            # which is impossible....
            if(len(x) == 0):    
                biz_features[idx,(1024*(kn)):(1024*(kn+1))] = np.zeros([1,1024])
            else:
                x = np.mean(x,axis=0)
                x = x.reshape([1,1024])
                biz_features[idx,(1024*(kn)):(1024*(kn+1))] = x

    return biz_features

In [10]:
for num_cluster in [2,3,4,5]:
    print "num_cluster = " + str(num_cluster)
    train_features, train_labels = shuffle(train_features, train_labels, random_state=7)
    
    kmn_holder = MiniBatchKMeans(n_clusters=num_cluster)
    kmn = kmn_holder.fit_predict(train_features, train_labels)
    kmn_train = kmn[:200000]
    kmn_test = kmn[200000:]
    
    tr_img_features, tr_labels, tr_indexes = get_data(train_features[:200000], train_labels[:200000])
    
    tr_biz_features = get_biz_features(tr_img_features, tr_labels, tr_indexes, num_cluster, kmn_train)
    
    ts_img_features, ts_labels, ts_indexes = get_data(train_features[200000:], train_labels[200000:])
    
    ts_biz_features = get_biz_features(ts_img_features, ts_labels, ts_indexes, num_cluster, kmn_test)
    
#     for idx, features in enumerate(img_features):
#         feature_index = indexes[idx]
#         cluster_lable = np.array(kmn_train[feature_index])
        
#         # for each biz group, mean feature vectore for those in the same cluster
#         for kn in range(num_cluster):
#             x = features[cluster_lable==kn]
#             # if feature doesn't belong to any cluster,
#             # which is impossible....
#             if(len(x) == 0):    
#                 biz_features[idx,(1024*(kn)):(1024*(kn+1))] = np.zeros([1,1024])
#             else:
#                 x = np.mean(x,axis=0)
#                 x = x.reshape([1,1024])
#                 biz_features[idx,(1024*(kn)):(1024*(kn+1))] = x
    
    lst_clf = train(2, tr_biz_features, tr_labels)
    
    test(lst_clf, ts_biz_features, ts_labels)

num_cluster = 2
training
Iteration: 0
Accuracy: 0.71 (+/- 0.38)
Accuracy: 0.79 (+/- 0.22)
Accuracy: 0.78 (+/- 0.24)
Accuracy: 0.76 (+/- 0.32)
Accuracy: 0.89 (+/- 0.15)
Accuracy: 0.80 (+/- 0.33)
Accuracy: 0.84 (+/- 0.28)
Accuracy: 0.76 (+/- 0.31)
Accuracy: 0.76 (+/- 0.25)
Iteration: 1
Accuracy: 0.71 (+/- 0.38)
Accuracy: 0.79 (+/- 0.22)
Accuracy: 0.78 (+/- 0.24)
Accuracy: 0.76 (+/- 0.32)
Accuracy: 0.89 (+/- 0.15)
Accuracy: 0.80 (+/- 0.33)
Accuracy: 0.84 (+/- 0.28)
Accuracy: 0.76 (+/- 0.30)
Accuracy: 0.76 (+/- 0.25)
0's classifier, test acc = 0.9481
1's classifier, test acc = 0.9481
2's classifier, test acc = 0.9457
3's classifier, test acc = 0.9506
4's classifier, test acc = 0.9704
5's classifier, test acc = 0.9802
6's classifier, test acc = 0.9704
7's classifier, test acc = 0.9136
8's classifier, test acc = 0.9457
num_cluster = 3
training
Iteration: 0
Accuracy: 0.79 (+/- 0.25)
Accuracy: 0.73 (+/- 0.34)
Accuracy: 0.74 (+/- 0.34)
Accuracy: 0.72 (+/- 0.31)
Accuracy: 0.88 (+/- 0.16)
Accurac